# Python Programming  and Row Operations in Python

In class, you have been studying row reduction techniques. In today's lab, you will use the small routines that you wrote on the last homework to implement algorithms for row reduction. We will use these routines in various ways throughout the semester, so it is important that you write these well (including good commenting!).

While this lab class is dedicated to implementing and applying ideas from your Linear Algebra class using Python, we will also be examining some of the problems you may encounter in converting math to code. Knowing these issues (and finding ways around them) will be central to your abilities to write code that does what you actually want it to! We'll start with that today.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

### Adding (and its Pitfalls), with *for* loops

**Question 1** Mathematically, an operation is *commutative* if doing it in reverse does not change the result. For example, addition of numbers is commutative, since $a+b=b+a$. Give an example of two matrices $A$ and $B$ such that $AB\neq BA$. This shows that matrix multiplication is not commutative.

**Question 2**  Is the dot product of two vectors a commutative operation? Explain. Suppose that $v$ and $w$ are two vectors of length $n$. Write their dot product using summation ($\Sigma$) notation.

Since adding numbers is commutative, it should not matter whether we add the products of numbers in a dot product forward or backward. However, we will see that this leads to problems when we try to calculate on a computer.

**Question 3** Suppose we have a vector of length *n* consisting of all ones, and another vector *v* of the same length. What does taking the dot product of these two vectors give?

**Question 4** Write code with no loops that assigns the reciprocals of the integers between 1 and a variable *n* to a vector *l*. Make sure it is a floating point vector! Note that if *l* is a vector, the code `l[::-1]` gives the same vector, but in reverse. For `n=10`, take your vector and sum it up in four different ways:
* By taking the dot product of *l* with a vector of ones of the same length.
* By taking the dot product of *l* in reverse order with a vector of ones of the same length.
* By running `np.sum()` on *l*.
* By running `np.sum()` on *l* in reverse order.

Now do the same for $n=10^7$.

In [1]:
import numpy as np
A = np.array([[3,4,5],[6,7,8],[9,10,11]])
B = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(A@B)
print(B@A)

c = np.array([2,3,4])
d = np.array([4,5,6])
print(c@d)
print(d@c)

n = 4
x = np.ones(n)
#v = np.ones(n)
v = np.array([2,4,6,8])
print(x@v)

n = 10
l = 1/(np.arange(1,11))
l1 = l@np.ones_like(l) #gets same shape as (l) 
l2 = l[::-1]@np.ones_like(l)
l3 = np.sum(l)
l4 = np.sum(l[::-1])
print(l)
print(l1)
print(l2)
print(l3)
print(l4)

[[ 54  66  78]
 [ 90 111 132]
 [126 156 186]]
[[ 42  48  54]
 [ 96 111 126]
 [150 174 198]]
47
47
20.0
[1.         0.5        0.33333333 0.25       0.2        0.16666667
 0.14285714 0.125      0.11111111 0.1       ]
2.9289682539682538
2.9289682539682538
2.9289682539682538
2.9289682539682538


#### What just happened?

It seems that adding numbers backward and forward seems to give different answers. While the differences are small, that is not really relevant: this is math. A small difference is still a difference. We need to understand what is going on if we want our code to give correct answers. What you saw above is an example of *floating point error*. 

You may read more about numerical representation and floating point errors at the links below:

* Binary representation: [here](http://ryanstutorials.net/binary-tutorial/) and [here](https://learn.sparkfun.com/tutorials/binary).
* Floating point error: [here](https://stackoverflow.com/questions/2100490/floating-point-inaccuracy-examples) and [here](https://accu.org/index.php/journals/1702).

The problem is most apparent when we add numbers of very different magnitudes, but can also be introduced when we divide by a number near zero. We will see that this has significant implications when row-reducing matrices if the entries in them are not of (approximately) the same magnitude. We will find ways around this in some case, starting on today's homework. Note that the internal routines (like `np.sum()`) generally give better answers than repeated manual addition, but even they are not precise - they just cannot be.

## Basic Row Operations in Python

The three basic row operations on matrices are:

* Swapping two rows.
* Multiplying a row of a matrix by a number;
* Adding (a multiple of) one row to another;

You wrote routines or one-liners for these on the last homework. You will show your code works briefly below, then combine these in another routine that will row-reduce matrices for us using the Gauss and Gauss-Jordan algorithms.

**Question 5** Copy your routines or one-liners from your homework into the code box below. Test them by doing the following operations (in the order given) on the matrix
$$A=\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}$$
* Multiply the second row by 0.5;
* Add the third row to the first;
* Swap the second and third rows.

You probably want to do this by hand before testing your routines on a computer, just to make sure the answer is correct! Make sure your matrices are floating point, not integer!

In [3]:
def swaprows(A,i,j):
    A[[i, j],:] = A[[j, i],:] 
def rowmult(A,i,c):
    A[i] = A[i]*c
def rowaddmult(A,i,j,c):
    A[j] = A[i]*c + A[j]

A = np.arange(1.,10).reshape((3,3))
rowmult(A,1,1/2)
print(A)
rowaddmult(A,2,0,1)
print(A)
swaprows(A,1,2)
print(A)

[[1.  2.  3. ]
 [2.  2.5 3. ]
 [7.  8.  9. ]]
[[ 8.  10.  12. ]
 [ 2.   2.5  3. ]
 [ 7.   8.   9. ]]
[[ 8.  10.  12. ]
 [ 7.   8.   9. ]
 [ 2.   2.5  3. ]]


## Gaussian Row Reduction

Now that we have routines to implement the three elementary row operations on matrices, we can implement a routine to do row reduction for matrices. You have done a bunch of row reductions in class, but just for practice, here are a couple of examples. Do them by hand. In case you have already covered it in class, please do not use any sort of pivoting (row swaps) here. We will add pivoting in on the homework.

**Question 6** Row reduce the following matrices to echelon form by hand:
$$\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 10
\end{bmatrix}
\mbox{ and }
\begin{bmatrix}
2 & 4 & 10 & 4\\
1 & 7 & 5 & -9\\
-4 & 2 & 7 & -10 \\
-1 & 2 & 3 & -4
\end{bmatrix}$$

**Question 7** Write a routine called `rowred(A)` that takes a matrix $A$, make a copy of it, and process the copy to returns its row reduced form. Your routine should use the mini-routines or one-liners you developed on the last homework to implement the elementary row operations. Test your code on the two matrices in the question above. 

In [4]:
A = np.array([[1,2,3,4],[4,5,6,7],[7,8,10,11]])
#A = np.array([[2.,4,10,4],[1,7,5,-9],[-4,2,7,-10],[-1,2,3,-4]])

def rowred(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            rowaddmult(copy,pivotrow,i,((-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])))
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    return copy;

print(A)
print(rowred(A))

[[ 1  2  3  4]
 [ 4  5  6  7]
 [ 7  8 10 11]]
[[ 1  2  3  4]
 [ 0 -3 -6 -9]
 [ 0  0  1  1]]


On the homework, we will see that line-by-line row reduction sometimes hits problems with non-existent pivots or floating point errors and develop a strategy to fix this.

## Back Substitution

**Question 8** Working on paper, solve the following system of linear equations using row reduction of the corresponding augmented matrices to echelon form and using back-substitution:
$$\begin{array}
4x_1 + 6x_2 - x_3 + 2x_4 & = & 22\\
-x_1 + 9x_2 + 7x_3 - 6x_4 & = & -26\\
2x_1 + x_2 + 4x_3 - 2x_4 & = & -20\\
9x_1 + 6x_2 + 3x_3 - 7x_4 & = & -34\\
\end{array}$$

**Question 9** Note that when doing back-substitution using echelon form, in each step, you are subtracting particular numbers from the entries in your augmented columns. The quantities you are subtracting can be expressed as dot products. Your teacher will outline this idea. Take careful notes on it.

**Question 10** Write a function called `backsub(U,v)` that takes an upper triangular matrix *U* and a vector *v* and implements back-substitution using dot products to solve the equation $U\vec{x}=\vec{v}$. Be sure to test your code.

In [5]:
A = np.array([[1.,2,3],[0,-3,-5],[0,0,10./3]])
U = rowred(A)
v = np.array([5.,-6,0])
print(U)

def backsub(U,v):
    rows,cols = U.shape
    x = np.zeros(shape=(cols,1))
    for i in range (rows-1,-1,-1):
        for j in range (i+1,rows):
            x[i] += (U[i,j]*x[j])
        x[i] = v[i] - x[i]
        x[i] = x[i]/U[i,i]
    return x;

print(backsub(U,v))

def backsub(U,v):
    rows,cols = U.shape
    x = np.zeros(cols)
    for i in range (rows-1,-1,-1):
        x[i] += (U[i, i+1:rows]@x[i+1:rows]) # dot product version 
        #x[i] = (np.sum((U[i, i+1:rows])*x[i+1:rows])) #np.sum() version 
        #**note - it's important that np sees this as a 1d (row) vector, 
        # so that when we do asterisk multiplication it uses the correct values.
        x[i] = v[i] - x[i]
        x[i] = x[i]/U[i,i]
    return x;

print(backsub(U,v))


[[ 1.          2.          3.        ]
 [ 0.         -3.         -5.        ]
 [ 0.          0.          3.33333333]]
[[1.]
 [2.]
 [0.]]
[1. 2. 0.]


**Question 11** Let *A* be an $n\times n$ matrix and *v* a vector of length $n$. By creating an augmented matrix $[A|v]$ and running your `rowred()` and `backsub()` functions, explain how to solve the system $Ax=v$. Test this on the system of simultaneous equations from Question 8.

$$\begin{array}
4x_1 + 6x_2 - x_3 + 2x_4 & = & 22\\
-x_1 + 9x_2 + 7x_3 - 6x_4 & = & -26\\
2x_1 + x_2 + 4x_3 - 2x_4 & = & -20\\
9x_1 + 6x_2 + 3x_3 - 7x_4 & = & -34\\
\end{array}$$

In [6]:
a = np.array([[1.,6,-1,2],[-1,9,7,-6],[2,1,4,-2],[9,6,3,-7]])
v = np.array([22,-26,-20,-34])
a = np.array([])
#we have to place A and v into an augmented matrix so we can put this into REF together
#and then take the matrix and the vector apart again and do back substitution.
print(a)
print(v)
rows,cols = a.shape #dimensions of the original matrix yet to be augmented
A = np.zeros(rows*(cols+1)).reshape(rows,cols+1)
A[:rows,:cols] = a[:,:]
A[:,cols] = v
print(A)
sol = rowred(A)
newsol = backsub(sol[:4,:4],sol[:,4]) #here we have to separate the REF matrix into its augmented parts.
print(newsol)
print(a@newsol)


[[ 1.  6. -1.  2.]
 [-1.  9.  7. -6.]
 [ 2.  1.  4. -2.]
 [ 9.  6.  3. -7.]]
[ 22 -26 -20 -34]
[[  1.   6.  -1.   2.  22.]
 [ -1.   9.   7.  -6. -26.]
 [  2.   1.   4.  -2. -20.]
 [  9.   6.   3.  -7. -34.]]
[-1.  2. -3.  4.]
[ 22. -26. -20. -34.]


## A Quick Note on Computation and Comprehension

It is important for you as Linear Algebra students to know how to reduce matrices and do back substitution by hand. The fact that you have coded up your own routines to automate this task is a significant achievement, but does not diminish the importance of working by hand. Remember: you coded the algorithm correctly *because you understood how it works through hand computation*. Therefore, the code is not a magic box to you: you can read it, understand it, and use it correctly. As you code more sophisticated tools, be sure to always understand how they work. Try to implement algorithms yourself. It is a great way to gain deep understanding of what is really going on!